In [42]:
from pathlib import Path
import os, shutil, json, re, time
from typing import Optional, List

from dotenv import load_dotenv
from rapidfuzz import fuzz

from pydantic import BaseModel, Field, ValidationError

from langchain.llms              import OpenAI
from langchain.embeddings        import OpenAIEmbeddings
from langchain.text_splitter     import CharacterTextSplitter
from langchain.schema            import Document, BaseRetriever
from langchain_community.vectorstores import Chroma
from langchain.retrievers        import BM25Retriever, EnsembleRetriever
from langchain.memory            import ConversationBufferMemory
from langchain.agents            import Tool, AgentType, initialize_agent
from langchain.prompts           import PromptTemplate
from langchain.chains            import LLMChain
from langchain.output_parsers    import PydanticOutputParser
from langchain_core.exceptions   import OutputParserException     
import chromadb
from chromadb.config import Settings


In [43]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

persist_dir = str(Path.home() / "chroma_asic_idx")
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)


In [44]:
product_texts = [
    # ——— S19 Pro ———
    """
Bitmain Antminer S19 Pro 110 TH/s
Алгоритм SHA-256 (Bitcoin/BCH)
110 TH/s ±3 % • 3250 Вт ±5 % • 29,5 J/TH
Шум 75 дБ • 400×195×290 мм, 13,2 кг
Цена 199 000 ₽  (скидка от 3 шт)
Гарантия 12 мес. от Bitmain
Доставка СДЭК РФ / самовывоз (Москва)
    """,
    # ——— M30S++ ———
    """
MicroBT Whatsminer M30S++ 112 TH/s
SHA-256 • 112 TH/s ±2 % • 3472 Вт • 31 J/TH
Состояние БУ 2023, 1000 ч • гарантия 3 мес
Цена 128 000 ₽ • скидка 5 % ≥ 5 шт
Оплата BTC, Сбер, Tinkoff
Доставка Boxberry, ПЭК
    """,
    # ——— iPollo V1 Mini ———
    """
iPollo V1 Mini ETC 300 MH/s (Wi-Fi)
EtHash ETC • 300 MH/s ±10 % • 240 Вт
Шум 50 дБ • 178×143×90 мм, 2,1 кг
Цена 38 500 ₽ • гарантия 6 мес iPollo
Оплата USDT (TRC-20), карта РФ
Доставка EMS, Boxberry — в день оплаты
    """,
]
documents = [Document(page_content=t.strip()) for t in product_texts]


In [45]:
splitter  = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=150)
chunks    = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    chunks,
    embedding         = embeddings,
    collection_name   = "asic_store",
    persist_directory = persist_dir,
    client_settings   = Settings(anonymized_telemetry=False),
)

In [66]:
bm25 = BM25Retriever.from_documents(documents, k=3)

class FuzzyRetriever(BaseRetriever):
    model_config = {"extra": "allow"}
    _docs: List[Document] = bm25.docs
    _k:   int             = 3
    def _get_relevant_documents(self, query, *, run_manager=None, **kw):
        ranked = sorted(
            self._docs,
            key=lambda d: fuzz.partial_ratio(query.lower(), d.page_content.lower()),
            reverse=True,
        )
        return ranked[: self._k]
    async def _aget_relevant_documents(self, query, *, run_manager=None, **kw):
        return self._get_relevant_documents(query)

hybrid_retriever = EnsembleRetriever(
    retrievers=[
        vectorstore.as_retriever(search_kwargs={"k": 8}),
        bm25,
        FuzzyRetriever(),
    ],
    weights=[0.5, 0.35, 0.15],
)


In [67]:
class CustomerCard(BaseModel):
    name:        Optional[str] = Field(None, description="Имя клиента")
    telegram:    Optional[str] = Field(None, description="Телеграм @...")
    phone:       Optional[str] = Field(None, description="Телефон")

    location:    Optional[str] = Field(None, description="Город/регион")
    entity_type: Optional[str] = Field(None, description="Физ/Юр лицо, компания")
    mining_years:Optional[int] = Field(None, description="Опыт майнинга, годы")
    rigs_owned:  Optional[int] = Field(None, description="Сколько асиков уже есть")
    rigs_plan:   Optional[int] = Field(None, description="Сколько планирует купить")

    has_asic:    Optional[bool]  = None
    electricity_price: Optional[float] = Field(None, description="₽/кВт⋅ч")
    budget:      Optional[int]  = Field(None, description="Бюджет ₽")
    host_choice: Optional[str]  = Field(None, description="Размещение: свой хостинг / наш")
    free_power:  Optional[int]  = Field(None, description="Свободные кВт")

    knowledge_lvl:        Optional[int] = Field(None, description="1-10 оценка знаний")
    financial_potential:  Optional[int] = Field(None, description="1-10 потенциал")
    stage_closed:         Optional[bool] = None  # сделка закрыта?

card_parser = PydanticOutputParser(pydantic_object=CustomerCard)
card_prompt  = PromptTemplate(
    template=(
        "Заполни карточку клиента по новой реплике.\n"
        "Текущий JSON: {current_json}\n\n"
        "Реплика: \"{utterance}\"\n\n"
        "Ответь ТОЛЬКО JSON:\n{fmt}"
    ),
    input_variables=["current_json", "utterance", "fmt"],
)
card_chain = LLMChain(llm=llm, prompt=card_prompt, output_parser=card_parser)

client_card = CustomerCard()

def update_card(cur_json: str, utterance: str, retry=2) -> CustomerCard:
    for _ in range(retry):
        try:
            raw = card_chain.invoke({
                "current_json": cur_json,
                "utterance":    utterance,
                "fmt": card_parser.get_format_instructions(),
            })["text"]
            if isinstance(raw, CustomerCard):
                return raw
            return CustomerCard(**raw) if isinstance(raw, dict) else CustomerCard.parse_raw(raw)
        except (OutputParserException, ValidationError):
            time.sleep(0.3)
    return CustomerCard.parse_raw(cur_json)

def classify_financial(card: CustomerCard) -> None:
    if card.financial_potential:
        return
    rigs = (card.rigs_plan or 0) + (card.rigs_owned or 0)
    if rigs >= 3000:  fp = 10
    elif rigs >= 1000: fp = 9
    elif rigs >= 500:  fp = 8
    elif rigs >= 100:  fp = 7
    elif rigs >= 50:   fp = 6
    elif rigs >= 20:   fp = 5
    elif rigs >= 10:   fp = 4
    elif rigs >= 7:    fp = 3
    elif rigs >= 4:    fp = 2
    elif rigs >= 1:    fp = 1
    else:              fp = None
    card.financial_potential = fp


In [69]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

class Intent(str, Enum):
    list   = "list"   
    detail = "detail"  
    budget = "budget" 

intent_parser = EnumOutputParser(enum=Intent)

intent_prompt = PromptTemplate(
    template=(
        "К какой категории относится запрос клиента?\n"
        "Варианты: list | detail | budget\n\n"
        "Запрос: \"{q}\"\n\n"
        "{format_instr}"         
    ),
    input_variables=["q", "format_instr"],
)

intent_chain = LLMChain(
    llm=llm,
    prompt=intent_prompt,
    output_parser=intent_parser,
)

In [71]:
def product_info(question: str) -> str:
    intent = intent_chain.invoke({
        "q": question,
        "format_instr": intent_parser.get_format_instructions(),  
    })["text"].strip()

    if intent == "list":
        return "В наличии:\n" + "\n".join(d.page_content.splitlines()[0] for d in documents)

    if intent == "budget":
        m = re.search(r"(\d[\d\s]{3,})", question)
        if not m:
            return "Уточните цифру бюджета в рублях."
        budget = int(m.group(1).replace(" ", ""))
        fits = [d for d in documents
                if (p := re.search(r"Цена\s+(\d[\d\s]+)", d.page_content))
                and int(p.group(1).replace(" ", "")) <= budget]
        return "Подходит:\n" + "\n".join(d.page_content.splitlines()[0] for d in fits) if fits else "За указанный бюджет моделей нет."

    docs = hybrid_retriever.invoke(question)
    if not docs:
        return "Информация не найдена."
    ctx = "\n---\n".join(d.page_content for d in docs)
    return llm.invoke(f"Ответь фактами:\n{ctx}\n\nВопрос: {question}\nОтвет:")


In [72]:
stage_prompt = PromptTemplate.from_template(
    """Выбери стадию: 1-имя/локация 2-потребности 3-презентация 4-закрытие.
Карта: {facts}
История: {chat_history}
Только цифра:""")
stage_chain = LLMChain(llm=llm, prompt=stage_prompt)

def missing(card: CustomerCard, st: str) -> str:
    need=[]
    if st=="1":
        if not card.name: need.append("имя")
        if not card.location: need.append("локация/размещение")
    elif st=="2":
        if card.has_asic is None: need.append("есть ли ASIC-ы")
        if card.has_asic and card.electricity_price is None: need.append("цена электричества")
        if (card.has_asic is False) and card.budget is None: need.append("бюджет или интересующие модели")
    elif st=="4" and card.contact is None:
        need.append("контакт и удобное время звонка")
    return ", ".join(need)

def build_agent(card: CustomerCard, st: str, memory):
    ask = missing(card, st)
    prefix = f"""
Ты — дружелюбный продавец ASIC-майнеров.

Текущий этап: {st}.
{('Выясни: '+ask) if ask else ''}

* Вопросы о товарах задавай через TOOL: product_info <вопрос>.
* Observation можно пересказывать своими словами либо частично,
    только если оно добавляет пользу клиенту.
* Не показывай системных тегов TOOL / Observation.
* Задавай открытые вопросы из скрипта (опыт, проблемы, хостинг, бюджет).
"""
    return initialize_agent(
        tools=[product_tool],
        llm=llm,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        agent_kwargs={"prefix": prefix},
    )


In [73]:
client_card = CustomerCard()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

print("🟢 Готов!  'exit' — вывести карточку.")
while True:
    user = input("\nВы: ")
    if user.lower() in {"exit", "quit"}:
        print("\n📇 Карточка клиента:")
        print(json.dumps(client_card.dict(exclude_none=True, ensure_ascii=False), indent=2))
        break

    print(f"👤 Клиент: {user}")

    client_card = update_card(client_card.model_dump_json(), user)
    classify_financial(client_card)
    memory.chat_memory.add_user_message(user)

    # ➜ стадия
    stage = stage_chain.invoke({
        "facts": client_card.model_dump_json(),
        "chat_history": memory.buffer,
    })["text"].strip()
    if stage not in {"1","2","3","4"}: stage="1"
    print(f"[Стадия: {stage}]")

    agent = build_agent(client_card, stage, memory)

    reply = agent.invoke({"input": user})["output"]
    print(f"🤖 Продавец: {reply}")
    
    client_card = update_card(client_card.model_dump_json(), reply)
    classify_financial(client_card)
    memory.chat_memory.add_ai_message(reply)


🟢 Готов!  'exit' — вывести карточку.
👤 Клиент: привет


/var/folders/dx/2j_jz8k12tn9dvfk_ny7sxr40000gn/T/ipykernel_44325/1260107577.py:56: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return CustomerCard.parse_raw(cur_json)


[Стадия: 1]


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Привет! Я могу помочь тебе с выбором ASIC-майнера. Какой у тебя бюджет?

> Finished chain.
🤖 Продавец: Привет! Я могу помочь тебе с выбором ASIC-майнера. Какой у тебя бюджет?
👤 Клиент: пока давай без бюджета
[Стадия: 4]


AttributeError: 'CustomerCard' object has no attribute 'contact'